# Conjugate Gradient for sparse linear systems

Author: Alexandre Gramfort

In [ ]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
from scipy import sparse

### Generate simulated data

In [ ]:
np.random.seed(0)
n_samples, n_features = 100, 1
X = np.random.randn(n_samples, n_features)
w = np.random.randn(n_features)
b = 10.
y = np.dot(X, w) + b
y += 0.3 * np.random.randn(n_samples)

In [ ]:
y.shape, X.shape

In [ ]:
X = sparse.csr_matrix(X)  # make X sparse
X

### Let's define a linear operator that implements a matrix vector product

In [ ]:
from scipy.sparse.linalg import LinearOperator

alpha = 0.  # the regularization parameter

def matvec(w):
    """Define the matrix vector product with X.T @ X + alpha Id"""
    return X.T.dot(X.dot(w)) + alpha * w
    
A = LinearOperator((n_features, n_features), matvec=matvec, dtype=X.dtype)

# A now behaves as matrix that can be multiplied by a vector
A @ np.array([1])

We can now use it to solve the Ridge regression problem with conjugate gradient.

What we need to solve is the following linear system:

$$(X^\top X + \alpha I) w = X^\top y$$

In [ ]:
def sparse_ridge(X, y, alpha=0., x0=None):
    n_features = X.shape[1]
    # matvec = lambda w: X.T.dot(X.dot(w)) + alpha * w
    def matvec(w):
        return X.T.dot(X.dot(w)) + alpha * w
    A = LinearOperator((n_features, n_features), 
                       matvec=matvec, dtype=X.dtype)
    w_hat, info = sparse.linalg.cg(A, X.T.dot(y), x0=x0)
    return w_hat

alpha = 0.  # the regularization parameter
w_hat = sparse_ridge(X, y)

In [ ]:
def plot_data(w, b=0.):
    plt.plot(X.toarray()[:, 0], y, 'o', alpha=0.2)

    xx = np.linspace(-2, 2, 100)
    yy = np.dot(xx[:, np.newaxis], w) + b
    plt.plot(xx, yy, 'k')
    plt.grid(True)

plot_data(w_hat)

You can observe that we have a problem with the regularization of the intercept.

Here are two ways to fix it.

First we will not regualize at all the intercept. We need to
create a new matrix that contains the last columns with ones.

In [ ]:
Xb = sparse.hstack((X, np.ones((n_samples, 1))))

def sparse_ridge_intercept(Xb, y, alpha=0., x0=None):
    n_features = Xb.shape[1] - 1
    def matvec(w):
        alpha_diag = np.zeros(n_features + 1)
        alpha_diag[:n_features] = alpha
        return Xb.T.dot(Xb.dot(w)) + np.diag(alpha_diag) @ w
    A = LinearOperator((n_features + 1, n_features + 1), 
                       matvec=matvec, dtype=Xb.dtype)
    (w_hat, b_hat), info = sparse.linalg.cg(A, Xb.T.dot(y), x0=x0)
    return w_hat, b_hat

alpha = 100.  # the regularization parameter
w_hat, b_hat = sparse_ridge_intercept(Xb, y, alpha=alpha, x0=[0, 0])

plot_data(w_hat, b_hat)

Option 2: try to not regularize the intercept too much.
For this the idea is to add a columns filled with a constant value.
If this value is 100 the intercept will be 100 times less regularized
that if it was filled with ones. This is related to the parameter
`intercept_scaling` in the [sklearn.linear_model.LogisticRegression](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html) model.

In [ ]:
Xb = sparse.hstack((X, 100 * np.ones((n_samples, 1))))

In [ ]:
alpha = 100.  # regularization parameter lambda
w_hat, b_hat = sparse_ridge(Xb, y, alpha=alpha)

In [ ]:
plot_data(w_hat, b_hat * 100)

## Let's do some "Big Data"

In [ ]:
n_samples, n_features = 10000, 1000000
density = 1e-5

rng1 = np.random.RandomState(42)
rng2 = np.random.RandomState(43)

nnz = int(n_samples*n_features*density)

row = rng1.randint(n_samples, size=nnz)
cols = rng2.randint(n_features, size=nnz)
data = rng1.rand(nnz)

X = sparse.coo_matrix((data, (row, cols)), shape=(n_samples, n_features))

In [ ]:
X.shape

In [ ]:
X.nnz

In [ ]:
w = np.random.randn(n_features)
y = X.dot(w)

In [ ]:
w_hat = sparse_ridge(X, y, alpha=0.01)

In [ ]:
w_hat.shape

### The benefits of warm start (providing a good init)

In [ ]:
%timeit sparse_ridge(X, y, alpha=0.01, x0=None)

In [ ]:
%timeit sparse_ridge(X, y, alpha=0.02, x0=w_hat)